In [9]:
from Crypto.Cipher import AES
from hashlib import sha1
import binascii
import codecs
import base64

In [10]:
def get_parity():
    """获取校验位"""
    a = [1,1,1,1,1,6]
    b = [7,3,1,7,3,1]
    c = 0
    for i in range(0,6):
        c = c + a[i]*b[i]
        parity = c % 10
    return parity

In [11]:
def parity_check(x):
    """调整奇偶校验"""
    k = []
    a = bin(int(x,16))[2:]
    for i in range(0,len(a),8):
        if (a[i:i+7].count("1"))%2 == 0:
            k.append(a[i:i+7])
            k.append('1') 
        else :
            k.append(a[i:i+7])
            k.append('0')      
    a1 = hex(int(''.join(k),2))
    return a1[2:]

In [12]:
if __name__ == "__main__":
    cipher_text = base64.b64decode(
        '9MgYwmuPrjiecPMx61O6zIuy3MtIXQQ0E59T3xB6u0Gyf1gYs2i3K9Jxaa0zj4gTMazJuApwd6+jdyeI5iGHvhQyDHGVlAuYTgJrbFDrfB22Fpil2NfNnWFBTXyf7SDI')
    IV = '00000000000000000000000000000000'

    # 求校验位
    parity = get_parity()
    print("parity = ",parity)

    # 得到机读信息
    MRZ_information = "12345678<81110182111116"+str(parity)

    # 求机读信息的 sha1 散列
    SHA1_MRZ_information = sha1(MRZ_information.encode()).hexdigest()

    # 截取前 16 字节得到 K_seed
    K_seed = SHA1_MRZ_information[:32]

    # 并置 K_seed 和 c 得到 D
    c = '00000001'
    D = K_seed + c

    # 计算 D 的 sha1 散列并形成密钥 K_a 和 K_b
    SHA1_D = sha1(codecs.decode(D,"hex")).hexdigest()
    K_a = SHA1_D[:16]
    K_b = SHA1_D[16:32]

    # 调整奇偶校验位并得到 K_enc
    K_a = parity_check(K_a)
    K_b = parity_check(K_b)
    K_enc = K_a + K_b
    print("K_enc = ",K_enc)

    # 解密
    m=AES.new(binascii.unhexlify(K_enc),AES.MODE_CBC,binascii.unhexlify(IV))
    plain_text = m.decrypt(cipher_text)
    print("plain_text = ", plain_text)

parity =  7
K_enc =  ea8645d97ff725a898942aa280c43179
plain_text =  b'Herzlichen Glueckwunsch. Sie haben die Nuss geknackt. Das Codewort lautet: Kryptographie!\x01\x00\x00\x00\x00\x00\x00'
